<a href="https://colab.research.google.com/github/10dimensions/grab_aiforsea/blob/master/aiforsea_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

url = 'https://raw.githubusercontent.com/10dimensions/grab_aiforsea/master/training.csv'
df = pd.read_csv(url, chunksize = 10000)

train_dataset = pd.DataFrame();
test_dataset = pd.DataFrame();

In [0]:
for chunk_df in df:
  
    chunk_df['day'] = chunk_df['day'] % 7
    chunk_df['timestamp'] = chunk_df['timestamp'].str.split(':').apply(lambda x: ( int(x[0])*60 + int(x[1])) / 10 )
    chunk_df['demand'] = chunk_df['demand'] * 1000
    
    train_chunk = chunk_df.sample(frac=0.8,random_state=0)
    test_chunk = chunk_df.drop(train_chunk.index)
    
    train_dataset = train_dataset.append( train_chunk )
    test_dataset = test_dataset.append( test_chunk )
    
    #chunk_result.to_csv(output_path, mode='a', header=False)

In [51]:
train_labels = train_dataset.pop('demand')
test_labels = test_dataset.pop('demand')

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

model = build_model()

W0616 17:00:00.511545 140104979425152 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [52]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

ValueError: ignored

In [0]:
loss, mae, mse = model.evaluate(train_dataset, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))


test_predictions = model.predict(test_dataset).flatten()
error = test_predictions - test_labels